In [1]:
cd("/g/data/v45/nc3020/Oceananigans-Tutorial-CFP/near-global/")

ENV["JULIA_DEPOT_PATH"] = "/g/data/v45/nc3020/.julia:/share/julia/site/"
ENV["JULIA_LOAD_PATH"] = "@:@v#.#:@stdlib:@site"

using Pkg; Pkg.activate(@__DIR__);

  Activating project at `/g/data/v45/nc3020/Oceananigans-Tutorial-CFP/near-global`


In [7]:
using ClimaOcean.NearGlobalSimulations: quarter_degree_near_global_simulation

using Oceananigans
using Oceananigans.Units
using Oceananigans.Utils: WallTimeInterval
using Oceananigans.BuoyancyModels: buoyancy
using Oceananigans.Models.HydrostaticFreeSurfaceModels: VerticalVorticityField
using JLD2

In [3]:
boundary_layer_turbulence_closure = RiBasedVerticalDiffusivity()

┌ Warning: RiBasedVerticalDiffusivity is an experimental turbulence closure that 
│ is unvalidated and whose default parameters are not calibrated for 
│ realistic ocean conditions or for use in a three-dimensional 
│ simulation. Use with caution and report bugs and problems with physics 
│ to https://github.com/CliMA/Oceananigans.jl/issues.
└ @ Oceananigans.TurbulenceClosures /g/data/v45/nc3020/.julia/packages/Oceananigans/KTw3g/src/TurbulenceClosures/turbulence_closure_implementations/ri_based_vertical_diffusivity.jl:83


RiBasedVerticalDiffusivity{VerticallyImplicitTimeDiscretization}
├── Ri_dependent_tapering: HyperbolicTangentRiDependentTapering
├── κ₀: 0.5
├── κᶜᵃ: 1.7
├── Cᵉⁿ: 0.1
├── Cᵃᵛ: 0.6
├── Ri₀: 0.1
└── Riᵟ: 0.4

In [8]:
start_time = 0days
stop_time  = start_time + 1years

architecture = GPU()

simulation = quarter_degree_near_global_simulation(architecture;
                                                   start_time, stop_time, boundary_layer_turbulence_closure)

[ Info: Reading initial conditions...
[ Info: ... read initial conditions (1.877 seconds)
[ Info: Reading boundary conditions...
[ Info: ... read boundary conditions (677.651 ms)
┌ Info: Created 1440×600×48 ImmersedBoundaryGrid{Float64, Periodic, Bounded, Bounded} on GPU with 5×5×5 halo:
│ ├── immersed_boundary: GridFittedBottom(min(h)=-1.05e+04, max(h)=1.00e+02)
│ ├── underlying_grid: 1440×600×48 LatitudeLongitudeGrid{Float64, Periodic, Bounded, Bounded} on GPU with 5×5×5 halo and with precomputed metrics
│ ├── longitude: Periodic λ ∈ [-180.0, 180.0) regularly spaced with Δλ=0.25
│ ├── latitude:  Bounded  φ ∈ [-75.0, 75.0]   regularly spaced with Δφ=0.25
└ └── z:         Bounded  z ∈ [-5244.5, 0.0]  variably spaced with min(Δz)=10.0, max(Δz)=410.5
┌ Info: ... built HydrostaticFreeSurfaceModel{GPU, ImmersedBoundaryGrid}(time = 0 seconds, iteration = 0)
│ ├── grid: 1440×600×48 ImmersedBoundaryGrid{Float64, Periodic, Bounded, Bounded} on GPU with 5×5×5 halo
│ ├── timestepper: QuasiAdamsB

Simulation of HydrostaticFreeSurfaceModel{GPU, ImmersedBoundaryGrid}(time = 0 seconds, iteration = 0)
├── Next time step: 6 minutes
├── Elapsed wall time: 0 seconds
├── Wall time per iteration: NaN years
├── Stop time: 1 year
├── Stop iteration : Inf
├── Wall time limit: Inf
├── Callbacks: OrderedDict with 5 entries:
│   ├── stop_time_exceeded => Callback of stop_time_exceeded on IterationInterval(1)
│   ├── stop_iteration_exceeded => Callback of stop_iteration_exceeded on IterationInterval(1)
│   ├── wall_time_limit_exceeded => Callback of wall_time_limit_exceeded on IterationInterval(1)
│   ├── nan_checker => Callback of NaNChecker for u on IterationInterval(100)
│   └── progress => Callback of (::ClimaOcean.NearGlobalSimulations.var"#progress#9"{Vector{UInt64}}) on IterationInterval(10)
├── Output writers: OrderedDict with no entries
└── Diagnostics: OrderedDict with no entries

KeyError: KeyError: key "usage_request" not found

In [5]:
# Define output
slices_save_interval = 1day
fields_save_interval = 30days
Nx, Ny, Nz = size(simulation.model.grid)

dir = pwd() * "/output"

closure_name = typeof(boundary_layer_turbulence_closure).name.wrapper
output_prefix = "near_global_$(Nx)_$(Ny)_$(Nz)_$closure_name"

"near_global_1440_600_48_RiBasedVerticalDiffusivity"

In [6]:
model = simulation.model

simulation.output_writers[:checkpointer] = Checkpointer(model; dir,
                                                        prefix = output_prefix * "_checkpointer",
                                                        schedule = WallTimeInterval(10minutes),
                                                        cleanup = true,
                                                        overwrite_existing = true)

simulation.output_writers[:fields] = JLD2OutputWriter(model, merge(model.velocities, model.tracers); dir,
                                                      schedule = TimeInterval(slices_save_interval),
                                                      filename = output_prefix * "_fields",
                                                      with_halos = true,
                                                      overwrite_existing = true)

slice_indices = [(:, :, Nz), (:, :, Nz-10)]
output_names = [:surface, :near_surface]

for n = 1:2
    indices = slice_indices[n]

    outputs = Dict()

    for name in keys(model.tracers)
        c = model.tracers[name]
        outputs[name] = Field(c; indices)
    end

    outputs[:u] = Field(model.velocities.u; indices)
    outputs[:v] = Field(model.velocities.v; indices)
    outputs[:w] = Field(model.velocities.w; indices)
    outputs[:η] = model.free_surface.η
    outputs[:ζ] = VerticalVorticityField(model.grid, model.velocities; indices)

    name = output_names[n]
    simulation.output_writers[name] = JLD2OutputWriter(model, outputs; dir,
                                                       schedule = TimeInterval(slices_save_interval),
                                                       filename = output_prefix * "_fields_$name",
                                                       with_halos = true,
                                                       overwrite_existing = true)
end

In [23]:
# start with small timestep

spinup = 7days
simulation.Δt = 2minute
stop_time = simulation.stop_time
simulation.stop_time = time(simulation) + spinup

@info "Running spinup simulation with Δt = $(prettytime(simulation.Δt)) until t = $(prettytime(spinup))"
run!(simulation)



# now increase timestep

simulation.stop_time = start_time + stop_time - spinup
simulation.Δt = 10minutes

@info "Running simulation with Δt = $(prettytime(simulation.Δt)) until t = $(prettytime(spinup))"
run!(simulation)

@info "Simulation took $(prettytime(simulation.run_wall_time))."

[ Info: Running spinup simulation with Δt = 2 minutes until t = 7 days
[ Info: Initializing simulation...
[ Info: Time:    0 seconds, iteration: 0, max(|u|): 0.00e+00 ms⁻¹, wmax: 0.00e+00, loc: (1, 1, 1), wall time: 4.988 seconds
[ Info:     ... simulation initialization complete (37.274 seconds)
[ Info: Executing initial time step...
[ Info:     ... initial time step complete (3.235 minutes).
[ Info: Time:   20 minutes, iteration: 10, max(|u|): 1.24e-01 ms⁻¹, wmax: 2.12e-03, loc: (1411, 85, 15), wall time: 4.118 minutes
[ Info: Time:   40 minutes, iteration: 20, max(|u|): 2.88e-01 ms⁻¹, wmax: 3.74e-03, loc: (1404, 82, 10), wall time: 2.796 seconds
[ Info: Time:       1 hour, iteration: 30, max(|u|): 4.42e-01 ms⁻¹, wmax: 5.22e-03, loc: (1411, 85, 15), wall time: 2.753 seconds
[ Info: Time:  1.333 hours, iteration: 40, max(|u|): 4.71e-01 ms⁻¹, wmax: 6.50e-03, loc: (1411, 85, 15), wall time: 2.752 seconds
[ Info: Time:  1.667 hours, iteration: 50, max(|u|): 6.68e-01 ms⁻¹, wmax: 7.68e-03,

LoadError: InterruptException:

KeyError: KeyError: key "usage_request" not found

In [37]:
saved_output_filename = output_prefix * "_fields_surface" * ".jld2"

"near_global_1440_600_48_RiBasedVerticalDiffusivity_fields_surface.jld2"

In [48]:
η_t = FieldTimeSeries("output/" * saved_output_filename, "η")
ζ_t = FieldTimeSeries("output/" * saved_output_filename, "ζ")

InterruptException: InterruptException:

In [45]:
size(ζ_t)

InterruptException: InterruptException:

In [46]:
λη, φη, zη = nodes(η_t[1])
λζ, φζ, zζ = nodes(ζ_t[1])

InterruptException: InterruptException:

In [47]:
using Oceananigans.ImmersedBoundaries: mask_immersed_field!

function mask_and_get_interior(φ_t, n)
    mask_immersed_field!(φ_t[n], NaN)
    return interior(φ_t[n], :, :, 1)
end

mask_and_get_interior (generic function with 1 method)

In [ ]:
n = Observable(1)

title = @lift @sprintf("t = %1.2f days = %1.2f T₂", round(times[$n]/day, digits=2) , round(times[$n]/T₂, digits=2))

ηₙ = @lift mask_and_get_interior(η_t, $n)
ζₙ = @lift mask_and_get_interior(ζ_t, $n)

axis_kwargs = (xlabel = "x [km]",
               ylabel = "z [m]",
               limits = ((-Lx/2e3, Lx/2e3), (-H, 0)),
               titlesize = 20)

ulim   = 0.5 * maximum(abs, u_t[end])
wlim   = maximum(abs, w_t[end])

fig = Figure(resolution = (1000, 900))

ax_η = Axis(fig[2, 1];
            title = "sea surface height", axis_kwargs...)

ax_ζ = Axis(fig[3, 1];
            title = "vorticity", axis_kwargs...)

fig[1, :] = Label(fig, title, fontsize=24, tellwidth=false)

hm_η = heatmap!(ax_η, λη, φη, ηₙ;
                colorrange = (-ηlim, ηlim),
                colormap = :balance)
Colorbar(fig[2, 2], hm_η)

hm_ζ = heatmap!(ax_ζ, λζ, φζ, ζₙ;
                colorrange = (-ζlim, ζlim),
                colormap = :balance)
Colorbar(fig[3, 2], hm_ζ)

fig